### MNIST 예측 모델
* 데이터 셋 : MNIST_test_csv, MNIST_train_csv
* 피쳐/속성 : 그림 파일
* 타겟/라벨 : 그림이 나타내는 번호
* 학습-방법 : 지도학습 > 분류 > 다중분류
* 알고리즘 : 인공신경망(ANN) -> MLP,DNN : 은닉층이 많은 구성
* 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비
<hr>

In [133]:
# 모듈 로딩
# - Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.classification import MulticlassF1Score
from torchinfo import summary

# - Data 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
import torch.optim.lr_scheduler as lr_scheduler

In [134]:
### 데이터 로딩
TRAIN_FILE = '../data/MNIST_train.csv'
TEST_FILE = '../data/MNIST_test.csv'

### CSV >> DF
MNIST_TRAIN_DF = pd.read_csv(TRAIN_FILE,header=None)
MNIST_TEST_DF = pd.read_csv(TEST_FILE,header=None)

In [135]:
MNIST_TRAIN_DF.head(3)
MNIST_TEST_DF.head(3)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
MNIST_TEST_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, 0 to 784
dtypes: int64(785)
memory usage: 59.9 MB


In [137]:
MNIST_TRAIN_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, 0 to 784
dtypes: int64(785)
memory usage: 59.9 MB


In [138]:
# Target column 값 확인하기
MNIST_TEST_DF[0].unique()

array([7, 2, 1, 0, 4, 9, 5, 6, 3, 8], dtype=int64)

In [139]:
class MNIST_Dataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF 
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
    
    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values) 
        
        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [140]:
# DF에서 피쳐와 타겟 추출
Train_featureDF = MNIST_TRAIN_DF.drop(columns=0)
Train_targetDF = MNIST_TRAIN_DF[0].to_frame()

Test_featureDF = MNIST_TEST_DF.drop(columns=0)
Test_targetDF = MNIST_TEST_DF[0].to_frame()

In [141]:
Train_targetDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       10000 non-null  int64
dtypes: int64(1)
memory usage: 78.2 KB


In [142]:
# 데이터 스케일링
standard_scaler = StandardScaler()
Train_standard_Scaled = standard_scaler.fit_transform(Train_featureDF)
Test_featureDF_Scaled = standard_scaler.fit_transform(Test_featureDF)

In [143]:
print(f'Train_standard_Scaled.shape => {Train_standard_Scaled.shape}')
print(f'Test_featureDF_Scaled.shape => {Test_featureDF_Scaled.shape}')

Train_standard_Scaled.shape => (10000, 784)
Test_featureDF_Scaled.shape => (10000, 784)


In [144]:
# - 커스텀데이터셋 인스턴스 생성
# MNIST_TRAIN_DS = MNIST_Dataset(Train_featureDF,Train_targetDF)
# MNIST_TEST_DS = MNIST_Dataset(Test_featureDF,Test_targetDF)

[4] 학습 준비 <hr>
- 학습_횟수 : EPOCH         <- 처음~끝까지 공부하는 단위
- 배치_크기 : BATCH_SIZE    <- 한번에 학습할 데이터셋 양 
- 위치_지정 : DEVICE    <- 텐서 저장 및 실행 위치 (GPU/CPU)
- 학_습_률 : LR 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001~0.1 

In [145]:
### 학습 진행 관련 설정
EPOCH = 100
BATCH_SIZE = 100
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.01

<hr>

In [146]:
class DynamicModel(nn.Module):

    # 모델 구조 설계 함수 즉, 생성자 메서드
    def __init__(self, in_in, in_out, out_out, h_ins=[], h_outs=[]):
        super().__init__()
        
        self.in_layer=nn.Linear(in_in, h_ins[0] if len(h_ins) else in_out )
        
        self.h_layers=nn.ModuleList()
        for idx  in range(len(h_ins)):
            self.h_layers.append( nn.Linear(h_ins[idx], h_outs[idx])  )
        
        self.out_layer=nn.Linear(h_outs[-1]  if len(h_outs) else in_out, out_out)
        
        
    # 학습 진행 콜백 메서드
    def forward(self, x): 
        # 입력층
        y=self.in_layer(x)                  # y=x1w1+x2w2+x3w3+b
        y=F.relu(y)                         # 0 <= y
        #y=F.relu(self.in_layer(x) )

        # 은닉층
        for linear in self.h_layers:
            y=linear(y)
            y=F.relu(y) 
            #y=F.relu( linear(y) )
            
        # 출력층
        return self.out_layer(y)    

In [147]:
# 모델 인스턴스 생성 50 70 90   70 90 110
h_ins_, h_outs_=[150,200,150], [200,150,100]
m1=DynamicModel(in_in=784, in_out=5, out_out=10, h_ins=h_ins_, h_outs=h_outs_)

In [148]:
# 모델 구조 확인
print(m1)

DynamicModel(
  (in_layer): Linear(in_features=784, out_features=150, bias=True)
  (h_layers): ModuleList(
    (0): Linear(in_features=150, out_features=200, bias=True)
    (1): Linear(in_features=200, out_features=150, bias=True)
    (2): Linear(in_features=150, out_features=100, bias=True)
  )
  (out_layer): Linear(in_features=100, out_features=10, bias=True)
)


In [149]:
# DS과 DL 인스턴스
#- 학습용, 검증용, 테스트용 데이터 분리
X_train, X_val, y_train, y_val =train_test_split(Train_standard_Scaled, Train_targetDF, random_state=1)


In [150]:
y_train.shape

(7500, 1)

In [151]:

X_train= pd.DataFrame(X_train)
X_val=pd.DataFrame(X_val)
y_train=pd.DataFrame(y_train)
y_val=pd.DataFrame(y_val)

In [152]:
trainDS=MNIST_Dataset(X_train, y_train)
valDS=MNIST_Dataset(X_val, y_val)
testDS=MNIST_Dataset(Test_featureDF, Test_targetDF)

print(f'{X_train.shape} {Test_featureDF.shape} {X_val.shape}')
print(f'{y_train.shape} {Test_featureDF.shape} {y_val.shape}')

# - 학습용 데이터로더 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

(7500, 784) (10000, 784) (2500, 784)
(7500, 1) (10000, 784) (2500, 1)


* 최적화, 손실함수 인스턴스 생성

In [153]:
# 최적화 인스턴스 => W,b 텐서를 전달
optimizer = optim.Adam(m1.parameters(),lr=LR)

# 최적화 스케쥴링 인스턴스 생성 => lr 조절 및 성능 개선 여부 체크
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=5, verbose=True) # score 사용 시
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=100, verbose=True)


# 손실함수 인스턴스 => 다중분류 CrossEntropyLoss
crossLoss = nn.CrossEntropyLoss()
# multi_F1 = MulticlassF1Score(num_classes=10)




c:\Users\KDP-14\anaconda3\envs\TORCH_38\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


[5] 학습 진행 <hr>

In [154]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

# 저장 경로
SAVE_PATH = '../models/MNIST/MCF/'
# 저장 파일명
SAVE_FILE = SAVE_PATH +'model_train_wbs.pth'
# 모델 구조 및 파라미터 모두 저장 파일명
SAVE_MODEL= SAVE_PATH +'model_all.pth'

In [155]:
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH) # 폴더/폴더/... 하위 폴더 까지 생성

In [156]:
for f, t in trainDL:
    print(f.shape, t.shape)
    break

torch.Size([100, 784]) torch.Size([100, 1])


In [157]:
# 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTROY=[[],[]], [[],[]]

## 학습 모니터링/스케줄링 설정 
# => LOSS_HISTORY, SCORE_HISTROY 활용
# => 임계기준 : 10번
BREAK_CNT = 0
LIMIT_VALUE = 10


for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    m1.train()

    # 배치크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total=0, 0
    for featureTS, targetTS in trainDL:

        # 학습 진행
        pre_y=m1(featureTS)

        # 손실 계산
        loss=crossLoss(pre_y, targetTS.reshape(-1).long())
        loss_total += loss.item()

        # 성능평가 계산
        score=MulticlassF1Score(num_classes=10)(pre_y, targetTS.reshape(-1))
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    m1.eval()
    with torch.no_grad():

        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)

        # 추론/평가
        pre_val=m1(val_featureTS)

        # 손실
        loss_val=crossLoss(pre_val, val_targetTS.reshape(-1).long())

        # 성능평가
        score_val=MulticlassF1Score(num_classes=10)(pre_val, val_targetTS.reshape(-1))
    
    # 에포크 당 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/len(trainDL))
    SCORE_HISTROY[0].append(score_total/len(trainDL))

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTROY[1].append(score_val)

    print(f'{epoch}/{EPOCH} => [TRAIN] LOSS: {LOSS_HISTORY[0][-1]} SCORE: {SCORE_HISTROY[0][-1]}')
    print(f'\t=> [VAL] LOSS: {LOSS_HISTORY[1][-1]} SCORE: {SCORE_HISTROY[1][-1]}')

    # 성능이 좋은 학습 가중치 저장
    SAVE_MODEL = f'loss({loss_total/len(trainDL):.5f})_score({score_total/len(trainDL):.5f}).pth'
    # SAVE_FILE = f'model_train_wbs_{epoch}_{score_val}.pth'
    if len(SCORE_HISTROY[1]) == 1:
        # # 첫번째라서 무조건 모델 파라미터 저장
        # torch.save(m1.state_dict(),SAVE_FILE)
        # 모델 전체 저장
        torch.save(m1,SAVE_PATH+SAVE_MODEL)
    
    else:
        if SCORE_HISTROY[1][-1] > max(SCORE_HISTROY[1][:-1]):
            # 두번째 ==> 첫번째보다 성능 좋으면 저장
            # torch.save(m1.state_dict(),SAVE_FILE)
            torch.save(m1,SAVE_PATH+SAVE_MODEL)
    # 최적화 스케쥴러 인스턴스 업데이트
    scheduler.step(loss_val)
    print(f'scheduler.num_bad_epochs => {scheduler.num_bad_epochs}', end=' ')
    print(f'scheduler.patience => {scheduler.patience}')

    # 손실 감소(또는 성능 개선)이 안되는 경우 조기종료
    if scheduler.num_bad_epochs >= scheduler.patience:
        print(f'{scheduler.patience}EPOCH 성능 개선이 없어서 조기종료함')
        break

0/100 => [TRAIN] LOSS: 0.6294927416245143 SCORE: 0.7974235946436723
	=> [VAL] LOSS: 0.3514135479927063 SCORE: 0.8981797695159912
scheduler.num_bad_epochs => 0 scheduler.patience => 100
1/100 => [TRAIN] LOSS: 0.2987623898188273 SCORE: 0.9096656680107117
	=> [VAL] LOSS: 0.3756755590438843 SCORE: 0.9175184965133667
scheduler.num_bad_epochs => 1 scheduler.patience => 100
2/100 => [TRAIN] LOSS: 0.24161204059918723 SCORE: 0.9311590464909871
	=> [VAL] LOSS: 0.40365928411483765 SCORE: 0.9062132239341736
scheduler.num_bad_epochs => 2 scheduler.patience => 100
3/100 => [TRAIN] LOSS: 0.22990323156118392 SCORE: 0.9378097462654114
	=> [VAL] LOSS: 0.41436585783958435 SCORE: 0.9193542003631592
scheduler.num_bad_epochs => 3 scheduler.patience => 100
4/100 => [TRAIN] LOSS: 0.2574780141313871 SCORE: 0.936433044274648
	=> [VAL] LOSS: 0.44723910093307495 SCORE: 0.9138697385787964
scheduler.num_bad_epochs => 4 scheduler.patience => 100
5/100 => [TRAIN] LOSS: 0.19003955890734991 SCORE: 0.9579088870684306
	=

KeyboardInterrupt: 